In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Trascrizione dell'audio Madrelingua Thai tramite il Modello **Wav2vec2.0 Thai**


In [ ]:
!pip install torch torchaudio transformers librosa soundfile
!apt-get install -y ffmpeg

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("automatic-speech-recognition", model="airesearch/wav2vec2-large-xlsr-53-th")

Device set to use cpu


In [ ]:
# Load model directly
from transformers import AutoProcessor, AutoModelForCTC

processor = AutoProcessor.from_pretrained("airesearch/wav2vec2-large-xlsr-53-th")
model = AutoModelForCTC.from_pretrained("airesearch/wav2vec2-large-xlsr-53-th")

In [ ]:
!pip install torchaudio transformers
!apt-get install -y ffmpeg

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [ ]:
import torchaudio
import torchaudio.transforms as T
from transformers import pipeline
import numpy as np

# Inizializza la pipeline ASR Thai
pipe = pipeline("automatic-speech-recognition", model="airesearch/wav2vec2-large-xlsr-53-th")

def prepara_audio(percorso_audio):
    # Carica e prepara l'audio
    waveform, sample_rate = torchaudio.load(percorso_audio)

    # Converte in mono se necessario
    if waveform.shape[0] > 1:
        waveform = waveform.mean(dim=0, keepdim=True)

    # Resample a 16 kHz
    if sample_rate != 16000:
        resampler = T.Resample(orig_freq=sample_rate, new_freq=16000)
        waveform = resampler(waveform)

    return waveform.squeeze().numpy()

# Percorsi degli audio
audio_corretto_path = "/content/drive/MyDrive/TesiMaggistrale/Lotus_dataset/PD/C/Clean/Wav/CCF001_Pa001/CCF001_Pa001_008.wav"
#audio_corrotto_path ="/content/drive/MyDrive/TesiMaggistrale/audiErrati/Corpus_InjectionOfError/C/Clean/Wav/CCF001_Pa001/CCF001_Pa001_008_corrupted.wav"
# Prepara gli audio
audio_corretto = prepara_audio(audio_corretto_path)
#audio_corrotto = prepara_audio(audio_corrotto_path)
# Trascrizione
trascrizione_corretto = pipe(audio_corretto)["text"]
#trascrizione_corrotto = pipe(audio_corrotto)["text"]
# Stampa dei risultati
print("Trascrizione (corretta):", trascrizione_corretto)
#print("Trascrizione (CORROTTA):", trascrizione_corrotto)

Device set to use cpu


Trascrizione (corretta): และ เพื่อให้ อัตราส่วน โอกาส ที่จะ มี ข้อมูล เป็น ลอจิก หนึ่ง  หรือ ลอจิก ศูนย์ เท่า เท่ากัน


**Iniezione di errori di pronuncia su fonemi e toni Thai**

In [ ]:
!pip install pythainlp

In [ ]:
import random
import nltk
from pythainlp.corpus.common import thai_words
# synsets: interfaccia al WordNet Thai, usata per ottenere insiemi di sinonimi (synset),  significati e relazioni semantiche tra le parole
from pythainlp.corpus.wordnet import synsets

# Assicura la presenza di WordNet
try:
    nltk.data.find('corpora/omw-1.4')
except LookupError:
    nltk.download('omw-1.4')

# Dizionario confusione esteso
pronunciation_confusions = {
    'light': {
        'consonants': {
            'ด': ['ต'], 'ต': ['ด'],
            'ร': ['ล'], 'ล': ['ร'],
            'บ': ['ป'], 'ป': ['บ'],
            'น': ['ม'], 'ม': ['น'],  # aggiunte
        },
        'vowels': {
            'า': ['ั'], 'ั': ['า'],
            'เ': ['แ'], 'แ': ['เ'],
            'ะ': ['า'], 'า': ['ะ'],  # aggiunte
        },
    },
    'heavy': {
        'consonants': {
            'บ': ['พ'], 'พ': ['บ'],
            'ช': ['ซ'], 'ซ': ['ช'],
            'ง': ['น'], 'น': ['ง'],
            'ก': ['ข', 'ค'], 'ข': ['ก'], 'ค': ['ก'],
            'ญ': ['ย'], 'ย': ['ญ'],  # aggiunte
        },
        'vowels': {
            'ิ': ['ี'], 'ี': ['ิ'],
            'ุ': ['ู'], 'ู': ['ุ'],
            'อ': ['โ'], 'โ': ['อ'],
        },
    },
    'tones': ['่', '้', '๊', '๋']
}

thai_vocab = set(thai_words())
tone_possible_chars = set("กขคฆงจชซฌญฎฏฐฑฒณดตถทธนบปผฝพฟภมยรฤลวสหฬอฮ")


def is_valid_lexical(word):
    return word in thai_vocab


def is_valid_semantic(word):
    return len(synsets(word)) > 0


def generate_pronunciation_variants(word, severity='light', max_steps=3):
    """
    Genera varianti di pronuncia di una parola simulando errori tipici.
    Garantisce che ogni rimozione di vocale o tono sia bilanciata da una sostituzione plausibile.
    """
    consonants = pronunciation_confusions[severity]['consonants']
    vowels = pronunciation_confusions[severity]['vowels']
    tones = pronunciation_confusions['tones']

    variants = set()

    def recursive_modify(current_word, steps_left, changes):
        if steps_left == 0:
            return

        for i, char in enumerate(current_word):
            new_variants = []

            # Sostituzione consonante
            if char in consonants:
                for rep in consonants[char]:
                    mod_word = current_word[:i] + rep + current_word[i+1:]
                    new_variants.append((mod_word, changes + [(char, rep, 'suono_consonante')]))

            # Sostituzione vocale
            if char in vowels:
                for rep in vowels[char]:
                    mod_word = current_word[:i] + rep + current_word[i+1:]
                    new_variants.append((mod_word, changes + [(char, rep, 'suono_vocale')]))

            # Inserimento di tono
            if char in tone_possible_chars:
                for tone in tones:
                    mod_word = current_word[:i+1] + tone + current_word[i+1:]
                    new_variants.append((mod_word, changes + [('', tone, 'aggiunta_tono')]))

            # Rimozione di tono o vocale → sostituzione alternativa se possibile
            if char in tones + list(vowels.keys()):
                # Prova a sostituire con un altro carattere simile
                if char in vowels:
                    alt_chars = [v for v in vowels[char] if v != char]
                    for alt in alt_chars:
                        mod_word = current_word[:i] + alt + current_word[i+1:]
                        new_variants.append((mod_word, changes + [(char, alt, 'sostituzione_vocale')]))
                elif char in tones:
                    alt_chars = [t for t in tones if t != char]
                    for alt in alt_chars:
                        mod_word = current_word[:i] + alt + current_word[i+1:]
                        new_variants.append((mod_word, changes + [(char, alt, 'sostituzione_tono')]))

            for variant, var_changes in new_variants:
                if variant not in variants:
                    variants.add((variant, tuple(var_changes)))
                    recursive_modify(variant, steps_left - 1, var_changes)

    recursive_modify(word, max_steps, [])
    return list(variants)


def inject_pronunciation_error(word, severity='light', max_steps=3):
    variants = generate_pronunciation_variants(word, severity=severity, max_steps=max_steps)

    # Filtra varianti valide e diverse dall'originale
    valid_variants = [(w, list(changes)) for (w, changes) in variants
                      if (is_valid_lexical(w) or is_valid_semantic(w)) and w != word]

    if valid_variants:
        chosen_word, changes = random.choice(valid_variants)
        return chosen_word, changes, True

    return word, [], False


def maybe_inject_pronunciation_multi_scaled(sentence, reference_sentence,
                                            base_inject_prob=0.2, severity='light', min_distance=1):
    """
    Simula errori di pronuncia in una frase:
    - Più modifiche distribuite nella frase, scalate con la lunghezza
    - Una sola modifica per parola
    - Riallinea la frase con gli spazi della reference_sentence
    - Distanza minima tra parole modificate
    """
    reference_words = reference_sentence.split()
    words = sentence.split()
    modified_words = []
    full_changes = []
    modified_flag = False

    last_modified_idx = -min_distance - 1

    # Calcolo probabilità scalata in base alla lunghezza della frase
    length = len(words)
    scaled_prob = base_inject_prob + 0.02 * length
    scaled_prob = min(scaled_prob, 0.5)  # massimo 50%

    for i, word in enumerate(words):
        # Evita cluster troppo vicini
        if i - last_modified_idx <= min_distance:
            modified_words.append(word)
            full_changes.append((word, word, [], False))
            continue

        word_severity = severity
        if random.random() < 0.3:
            word_severity = 'heavy' if severity == 'light' else 'light'

        # Decidi se modificare la parola
        if random.random() < scaled_prob:
            variants = generate_pronunciation_variants(word, severity=word_severity, max_steps=1)
            valid_variants = [(w, list(changes)) for (w, changes) in variants
                              if (is_valid_lexical(w) or is_valid_semantic(w)) and w != word]

            if valid_variants:
                chosen_word, changes = random.choice(valid_variants)
                applied = True
                modified_flag = True
                last_modified_idx = i
            else:
                chosen_word, changes, applied = word, [], False
        else:
            chosen_word, changes, applied = word, [], False

        modified_words.append(chosen_word)
        full_changes.append((word, chosen_word, changes, applied))

    # Riallineamento con spazi della reference_sentence
    new_sentence = ' '.join(modified_words[:len(reference_words)])
    return new_sentence, full_changes, modified_flag

# --- Esempio di utilizzo ---
sentence_no_spaces = "ฉัน รัก ภาษาไทย มาก"
reference_sentence = sentence_no_spaces

corrupted, error_log, modified = maybe_inject_pronunciation_multi_scaled(
    sentence_no_spaces,
    reference_sentence,
    base_inject_prob=0.2,
    severity='light',
    min_distance=1
)

print(" Originale:", sentence_no_spaces)
print(" Simulazione errori di pronuncia:", corrupted)

# Stampa log modifiche
modifiche_effettive = [entry for entry in error_log if entry[3] is True]
if not modifiche_effettive:
    print(" Nessuna modifica effettuata.")
else:
    for original, modified_word, changes, is_valid in modifiche_effettive:
        print(f"- '{original}' → '{modified_word}' Modificata")
        for old, new_c, err_type in changes:
            if old == '':
                print(f"    + Aggiunto tono: '{new_c}'")
            elif new_c == '':
                print(f"    - Rimosso '{old}' ({err_type})")
            else:
                print(f"    x {err_type.upper()} - '{old}' → '{new_c}'")



[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data]   Package omw is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


 Originale: ฉัน รัก ภาษาไทย มาก
 Simulazione errori di pronuncia: ฉัน รัก ภาษาไทย นาก
- 'มาก' → 'นาก' Modificata
    x SUONO_CONSONANTE - 'ม' → 'น'


***Pipline Completa!***

In [ ]:
import os
import csv
import torchaudio
import torchaudio.transforms as T
from transformers import pipeline
from pythaitts import TTS
from pythainlp import word_tokenize

# Percorsi
AUDIO_DIR = "/content/drive/MyDrive/TesiMaggistrale/audiErrati/U/Office/Wav/UOM046_Pa046"
OUTPUT_DIR = "/content/drive/MyDrive/TesiMaggistrale/audiErrati/Corpus_Injection_error_thai/"
CSV_PATH = os.path.join(OUTPUT_DIR, "pronunciation_error_dataset.csv")
os.makedirs(OUTPUT_DIR, exist_ok=True)

sr_pipe = pipeline("automatic-speech-recognition", model="airesearch/wav2vec2-large-xlsr-53-th")
tts_model = TTS(pretrained="khanomtan", version="1.0", mode="best_model")

# Funzione per preparare audio per ASR
def prepara_audio(path):
    waveform, sr = torchaudio.load(path)
    if waveform.shape[0] > 1:
        waveform = waveform.mean(dim=0, keepdim=True)
    if sr != 16000:
        waveform = T.Resample(sr, 16000)(waveform)
    return waveform.squeeze().numpy()

# Inizializza il CSV con intestazioni se non esiste
if not os.path.exists(CSV_PATH):
    with open(CSV_PATH, mode="w", encoding="utf-8", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["filename", "trascrizione_originale", "trascrizione_errata", "modificata", "dettagli_modifiche", "audio_path"])

# Trova file audio
audio_files = []
for root, _, files in os.walk(AUDIO_DIR):
    for file in files:
        if file.endswith(".wav"):
            audio_files.append(os.path.join(root, file))

print(f"Trovati {len(audio_files)} file audio.")

# Loop
for audio_path in audio_files:
    try:
        base_name = os.path.splitext(os.path.basename(audio_path))[0]

        # Step 1: ASR
        audio_data = prepara_audio(audio_path)
        trascrizione = sr_pipe(audio_data)["text"]
        frase_no_spazi = trascrizione.replace(" ", "")
        reference_sentence = frase_no_spazi

        # Step 1.1: Tokenizzazione Thai
        tokens = word_tokenize(frase_no_spazi, engine="newmm")
        frase_tokenizzata = " ".join(tokens)
        reference_sentence = frase_tokenizzata

        # Step 2: Iniezione errori
        frase_errata, log_error, modificato = maybe_inject_pronunciation_multi_scaled(
            frase_no_spazi,
            reference_sentence,
            base_inject_prob=0.2,
            severity='light',
            min_distance=1
        )

        # Step 3: TTS
        audio_out_path = os.path.join(OUTPUT_DIR, f"{base_name}_corrupted.wav")
        tts_model.tts(
            text=frase_errata,
            speaker_idx="Tsyncone",
            language_idx="th-th",
            return_type="file",
            filename=audio_out_path
        )

        # Step 4: Log dettagliato modifiche
        if not modificato:
            dettagli = "Nessuna modifica"
        else:
            modifiche_effettive = [entry for entry in log_error if entry[3] is True]
            dettaglio_lista = []
            for original_w, modified_w, changes, _ in modifiche_effettive:
                for old, new_c, tipo in changes:
                    if old == '':
                        dettaglio_lista.append(f"Aggiunto tono: '{new_c}' in '{modified_w}'")
                    elif new_c == '':
                        dettaglio_lista.append(f"Rimosso '{old}' da '{original_w}'")
                    else:
                        dettaglio_lista.append(f"{tipo.upper()} - '{original_w}': '{old}'→'{new_c}'")
            dettagli = " | ".join(dettaglio_lista) if dettaglio_lista else "Nessuna modifica"

        # Step 5: Salva nel CSV
        with open(CSV_PATH, mode="a", encoding="utf-8", newline="") as f:
            writer = csv.writer(f)
            writer.writerow([
                base_name,
                trascrizione,
                frase_errata,
                "SI" if modificato else "NO",
                dettagli,
                audio_out_path
            ])

        print(f"✅ Salvato: {base_name}")

    except Exception as e:
        print(f"❌ Errore su {audio_path}: {str(e)}")

print("✅ Creazione database completata.")


UNIONE DELLE DIVERSE CARTELLE

In [ ]:
import os
import pandas as pd

cartelle = [
    "/content/drive/MyDrive/TesiMaggistrale/audiErrati/C/Clean/Wav",
    "/content/drive/MyDrive/TesiMaggistrale/audiErrati/C/Clean/Wav/CCF002_Pa002",
    "/content/drive/MyDrive/TesiMaggistrale/audiErrati/C/Clean/Wav/CCF004_Pa004",
    "/content/drive/MyDrive/TesiMaggistrale/audiErrati/C/Clean/Wav/CCF005_Pa005",
    "/content/drive/MyDrive/TesiMaggistrale/audiErrati/C/Clean/Wav/CCF007_Pa007",
    "/content/drive/MyDrive/TesiMaggistrale/audiErrati/C/Clean/Wav/CCF008_Pa008",
    "/content/drive/MyDrive/TesiMaggistrale/audiErrati/C/Clean/Wav/CCF009_Pa009",
    "/content/drive/MyDrive/TesiMaggistrale/audiErrati/C/Clean/Wav/CCF011_Pa011",
    "/content/drive/MyDrive/TesiMaggistrale/audiErrati/C/Clean/Wav/CCF014_Pa014",
    "/content/drive/MyDrive/TesiMaggistrale/audiErrati/C/Clean/Wav/CCF017_Pa017",
    "/content/drive/MyDrive/TesiMaggistrale/audiErrati/C/Clean/Wav/CCF020_Pa020",
    "/content/drive/MyDrive/TesiMaggistrale/audiErrati/C/Clean/Wav/CCF024_Pa024",
    "/content/drive/MyDrive/TesiMaggistrale/audiErrati/C/Clean/Wav/CCM025_Pa025",
    "/content/drive/MyDrive/TesiMaggistrale/audiErrati/C/Clean/Wav/CCM027_Pa027",
    "/content/drive/MyDrive/TesiMaggistrale/audiErrati/C/Clean/Wav/CCM030_Pa030",
    "/content/drive/MyDrive/TesiMaggistrale/audiErrati/C/Clean/Wav/CCM032_Pa032",
    "/content/drive/MyDrive/TesiMaggistrale/audiErrati/C/Clean/Wav/CCM034_Pa034",
    "/content/drive/MyDrive/TesiMaggistrale/audiErrati/C/Clean/Wav/CCM036_Pa036",
    "/content/drive/MyDrive/TesiMaggistrale/audiErrati/C/Clean/Wav/CCM037_Pa037",
    "/content/drive/MyDrive/TesiMaggistrale/audiErrati/C/Clean/Wav/CCM040_Pa040",
    "/content/drive/MyDrive/TesiMaggistrale/audiErrati/C/Clean/Wav/CCM043_Pa043",
    "/content/drive/MyDrive/TesiMaggistrale/audiErrati/C/Clean/Wav/CCM044_Pa044",
    "/content/drive/MyDrive/TesiMaggistrale/audiErrati/C/Clean/Wav/CCM046_Pa046",
    "/content/drive/MyDrive/TesiMaggistrale/audiErrati/C/Clean/Wav/CCM047_Pa047",
    "/content/drive/MyDrive/TesiMaggistrale/audiErrati/C/Office/Wav/COF001_Pa001",
    "/content/drive/MyDrive/TesiMaggistrale/audiErrati/C/Office/Wav/COF002_Pa002",
    "/content/drive/MyDrive/TesiMaggistrale/audiErrati/C/Office/Wav/COF004_Pa004",
    "/content/drive/MyDrive/TesiMaggistrale/audiErrati/C/Office/Wav/COF005_Pa005",
    "/content/drive/MyDrive/TesiMaggistrale/audiErrati/C/Office/Wav/COF007_Pa007",
    "/content/drive/MyDrive/TesiMaggistrale/audiErrati/C/Office/Wav/COF008_Pa008",
    "/content/drive/MyDrive/TesiMaggistrale/audiErrati/C/Office/Wav/COF009_Pa009",
    "/content/drive/MyDrive/TesiMaggistrale/audiErrati/C/Office/Wav/COF011_Pa011",
    "/content/drive/MyDrive/TesiMaggistrale/audiErrati/C/Office/Wav/COF014_Pa014",
    "/content/drive/MyDrive/TesiMaggistrale/audiErrati/C/Office/Wav/COF017_Pa017",
    "/content/drive/MyDrive/TesiMaggistrale/audiErrati/C/Office/Wav/COF020_Pa020",
    "/content/drive/MyDrive/TesiMaggistrale/audiErrati/C/Office/Wav/COF024_Pa024",
    "/content/drive/MyDrive/TesiMaggistrale/audiErrati/C/Office/Wav/COM025_Pa025",
    "/content/drive/MyDrive/TesiMaggistrale/audiErrati/C/Office/Wav/COM027_Pa027",
    "/content/drive/MyDrive/TesiMaggistrale/audiErrati/C/Office/Wav/COM030_Pa030",
    "/content/drive/MyDrive/TesiMaggistrale/audiErrati/C/Office/Wav/COM032_Pa032",
    "/content/drive/MyDrive/TesiMaggistrale/audiErrati/C/Office/Wav/COM034_Pa034",
    "/content/drive/MyDrive/TesiMaggistrale/audiErrati/C/Office/Wav/COM036_Pa036",
    "/content/drive/MyDrive/TesiMaggistrale/audiErrati/C/Office/Wav/COM037_Pa037",
    "/content/drive/MyDrive/TesiMaggistrale/audiErrati/C/Office/Wav/COM040_Pa040",
    "/content/drive/MyDrive/TesiMaggistrale/audiErrati/C/Office/Wav/COM043_Pa043",
    "/content/drive/MyDrive/TesiMaggistrale/audiErrati/C/Office/Wav/COM044_Pa044",
    "/content/drive/MyDrive/TesiMaggistrale/audiErrati/C/Office/Wav/COM046_Pa046",
    "/content/drive/MyDrive/TesiMaggistrale/audiErrati/C/Office/Wav/COM047_Pa047",
    "/content/drive/MyDrive/TesiMaggistrale/audiErrati/U/Clean/Wav/UCF001_Pa001",
    "/content/drive/MyDrive/TesiMaggistrale/audiErrati/U/Clean/Wav/UCF002_Pa002",
    "/content/drive/MyDrive/TesiMaggistrale/audiErrati/U/Clean/Wav/UCF004_Pa004",
    "/content/drive/MyDrive/TesiMaggistrale/audiErrati/U/Clean/Wav/UCF005_Pa005",
    "/content/drive/MyDrive/TesiMaggistrale/audiErrati/U/Clean/Wav/UCF007_Pa007",
    "/content/drive/MyDrive/TesiMaggistrale/audiErrati/U/Clean/Wav/UCF008_Pa008",
    "/content/drive/MyDrive/TesiMaggistrale/audiErrati/U/Clean/Wav/UCF009_Pa009",
    "/content/drive/MyDrive/TesiMaggistrale/audiErrati/U/Clean/Wav/UCF011_Pa011",
    "/content/drive/MyDrive/TesiMaggistrale/audiErrati/U/Clean/Wav/UCF014_Pa014",
    "/content/drive/MyDrive/TesiMaggistrale/audiErrati/U/Clean/Wav/UCF017_Pa017",
    "/content/drive/MyDrive/TesiMaggistrale/audiErrati/U/Clean/Wav/UCF020_Pa020",
    "/content/drive/MyDrive/TesiMaggistrale/audiErrati/U/Clean/Wav/UCF024_Pa024",
    "/content/drive/MyDrive/TesiMaggistrale/audiErrati/U/Clean/Wav/UCM025_Pa025",
    "/content/drive/MyDrive/TesiMaggistrale/audiErrati/U/Clean/Wav/UCM027_Pa027",
    "/content/drive/MyDrive/TesiMaggistrale/audiErrati/U/Clean/Wav/UCM030_Pa030",
    "/content/drive/MyDrive/TesiMaggistrale/audiErrati/U/Clean/Wav/UCM032_Pa032",
    "/content/drive/MyDrive/TesiMaggistrale/audiErrati/U/Clean/Wav/UCM034_Pa034",
    "/content/drive/MyDrive/TesiMaggistrale/audiErrati/U/Clean/Wav/UCM036_Pa036",
    "/content/drive/MyDrive/TesiMaggistrale/audiErrati/U/Clean/Wav/UCM037_Pa037",
    "/content/drive/MyDrive/TesiMaggistrale/audiErrati/U/Clean/Wav/UCM040_Pa040",
    "/content/drive/MyDrive/TesiMaggistrale/audiErrati/U/Clean/Wav/UCM043_Pa043",
    "/content/drive/MyDrive/TesiMaggistrale/audiErrati/U/Clean/Wav/UCM044_Pa044",
    "/content/drive/MyDrive/TesiMaggistrale/audiErrati/U/Clean/Wav/UCM046_Pa046",
    "/content/drive/MyDrive/TesiMaggistrale/audiErrati/U/Clean/Wav/UCM047_Pa047",
    "/content/drive/MyDrive/TesiMaggistrale/audiErrati/U/Office/Wav/UOF001_Pa001",
    "/content/drive/MyDrive/TesiMaggistrale/audiErrati/U/Office/Wav/UOF002_Pa002",
    "/content/drive/MyDrive/TesiMaggistrale/audiErrati/U/Office/Wav/UOF004_Pa004",
    "/content/drive/MyDrive/TesiMaggistrale/audiErrati/U/Office/Wav/UOF005_Pa005",
    "/content/drive/MyDrive/TesiMaggistrale/audiErrati/U/Office/Wav/UOF007_Pa007",
    "/content/drive/MyDrive/TesiMaggistrale/audiErrati/U/Office/Wav/UOF008_Pa008",
    "/content/drive/MyDrive/TesiMaggistrale/audiErrati/U/Office/Wav/UOF009_Pa009",
    "/content/drive/MyDrive/TesiMaggistrale/audiErrati/U/Office/Wav/UOF011_Pa011",
    "/content/drive/MyDrive/TesiMaggistrale/audiErrati/U/Office/Wav/UOF014_Pa014",
    "/content/drive/MyDrive/TesiMaggistrale/audiErrati/U/Office/Wav/UOF017_Pa017",
    "/content/drive/MyDrive/TesiMaggistrale/audiErrati/U/Office/Wav/UOF020_Pa020",
    "/content/drive/MyDrive/TesiMaggistrale/audiErrati/U/Office/Wav/UOF024_Pa024",
    "/content/drive/MyDrive/TesiMaggistrale/audiErrati/U/Office/Wav/UOM025_Pa025",
    "/content/drive/MyDrive/TesiMaggistrale/audiErrati/U/Office/Wav/UOM027_Pa027",
    "/content/drive/MyDrive/TesiMaggistrale/audiErrati/U/Office/Wav/UOM030_Pa030",
    "/content/drive/MyDrive/TesiMaggistrale/audiErrati/U/Office/Wav/UOM032_Pa032",
    "/content/drive/MyDrive/TesiMaggistrale/audiErrati/U/Office/Wav/UOM034_Pa034",
    "/content/drive/MyDrive/TesiMaggistrale/audiErrati/U/Office/Wav/UOM036_Pa036",
    "/content/drive/MyDrive/TesiMaggistrale/audiErrati/U/Office/Wav/UOM037_Pa037",
    "/content/drive/MyDrive/TesiMaggistrale/audiErrati/U/Office/Wav/UOM040_Pa040",
    "/content/drive/MyDrive/TesiMaggistrale/audiErrati/U/Office/Wav/UOM043_Pa043",
    "/content/drive/MyDrive/TesiMaggistrale/audiErrati/U/Office/Wav/UOM044_Pa044",
    "/content/drive/MyDrive/TesiMaggistrale/audiErrati/U/Office/Wav/UOM046_Pa046",
    "/content/drive/MyDrive/TesiMaggistrale/audiErrati/U/Office/Wav/UOM047_Pa047",

]

csv_unificato = "/content/drive/MyDrive/TesiMaggistrale/audiErrati/merged_dataset2.csv"
df_totale = pd.DataFrame()

for cartella in cartelle:
    for root, _, files in os.walk(cartella):
        for file in files:
            if file == "pronunciation_error_dataset.csv":
                csv_path = os.path.join(root, file)
                df = pd.read_csv(csv_path)
                df_totale = pd.concat([df_totale, df], ignore_index=True)

df_totale.to_csv(csv_unificato, index=False)
print(f"✅ CSV unificato salvato in: {csv_unificato}")
